In [12]:
import pandas as pd

In [15]:
# Load Participant_Status table
Participant_Status = pd.read_csv('/home/mrudula/Downloads/Participant_Status_27Oct2024.csv')

# Load Demographics table
Demographics = pd.read_csv('/home/mrudula/Downloads//Demographics_27Oct2024.csv')

In [20]:


# Assuming you have already loaded your tables into pandas DataFrames
# Participant_Status and Demographics should be DataFrames that contain the data from those tables

# Step 1: Renaming the PATNO column in Participant_Status for clarity
participant_status = Participant_Status.rename(columns={"PATNO": "Participant_ID"})

# Step 2: Performing a join operation (similar to an SQL join) on the two DataFrames
combined_table = pd.merge(
    participant_status,
    Demographics,
    left_on="Participant_ID",
    right_on="PATNO",
    suffixes=("", "_drop")  # To differentiate columns if needed for duplicate names
)

# Step 3: Dropping unnecessary columns
columns_to_drop = [
    'HETERO', 'PANSEXUAL', 'ASEXUAL', 'OTHSEXUALITY', 
    'ORIG_ENTRY', 'LAST_UPDATE', 'HOWLIVE', 'GAYLES', 
    'PPMI_ONLINE_ENROLL', 'INEXPAGE', 'STATUS_DATE','COHORT_DEFINITION','REC_ID', 'PATNO' # This is redundant after the merge
]

combined_table.drop(columns=columns_to_drop, inplace=True)

# Step 4: Filtering rows based on the Enrollment Status column
valid_statuses = ['Enrolled', 'Complete', 'Withdrew']
combined_table = combined_table[combined_table['ENROLL_STATUS'].isin(valid_statuses)]
columns_to_drop = [
    'ENROLL_STATUS' # This is redundant after the merge
]

combined_table.drop(columns=columns_to_drop, inplace=True)
# Displaying the first few rows of the final DataFrame
combined_table.head()


,Participant_ID,COHORT,ENROLL_DATE,ENROLL_AGE,AV133STDY,TAUSTDY,GAITSTDY,PISTDY,SV2ASTDY,ENRLPINK1,...,BISEXUAL,HANDED,HISPLAT,RAASIAN,RABLACK,RAHAWOPI,RAINDALS,RANOS,RAWHITE,RAUNKNOWN
0,3000,2,02/2011,69.1,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,3001,1,03/2011,65.1,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,3002,1,03/2011,67.6,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,3003,1,04/2011,56.7,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,3004,2,04/2011,59.4,0.0,0.0,1.0,0.0,1.0,0.0,...,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [21]:
combined_table.columns

Index(['Participant_ID', 'COHORT', 'ENROLL_DATE', 'ENROLL_AGE', 'AV133STDY',
       'TAUSTDY', 'GAITSTDY', 'PISTDY', 'SV2ASTDY', 'ENRLPINK1', 'ENRLPRKN',
       'ENRLSRDC', 'ENRLHPSM', 'ENRLRBD', 'ENRLLRRK2', 'ENRLSNCA', 'ENRLGBA',
       'EVENT_ID', 'PAG_NAME', 'INFODT', 'AFICBERB', 'ASHKJEW', 'BASQUE',
       'BIRTHDT', 'SEX', 'CHLDBEAR', 'BISEXUAL', 'HANDED', 'HISPLAT',
       'RAASIAN', 'RABLACK', 'RAHAWOPI', 'RAINDALS', 'RANOS', 'RAWHITE',
       'RAUNKNOWN'],
      dtype='object')

In [23]:
import pandas as pd

# Assuming you have already loaded your Biospecimen_Analysis DataFrame
# Biospecimen_Analysis should be a DataFrame that contains the data from that table

Biospecimen_Analysis = pd.read_csv('/home/mrudula/Downloads/SAA_Biospecimen_Analysis_Results_27Oct2024.csv')
# Step 1: Filter for clinical event 'BL' and find the earliest RUNDATE for each PATNO
earliest_records = (
    Biospecimen_Analysis[Biospecimen_Analysis['CLINICAL_EVENT'] == 'BL']
    .groupby('PATNO', as_index=False)
    .agg(earliest_date=('RUNDATE', 'min'))  # Ensure 'RUNDATE' matches the actual column name
)

# Step 2: Merge the original DataFrame with the earliest_records DataFrame
Biospecimen_Analysis_Cleaned = pd.merge(
    Biospecimen_Analysis,
    earliest_records,
    left_on=['PATNO', 'RUNDATE'],  # Adjust if 'RUNDATE' is named differently
    right_on=['PATNO', 'earliest_date'],
    how='inner'  # This ensures we only keep records that match
)

# Displaying the cleaned DataFrame
Biospecimen_Analysis_Cleaned.head()


,PATNO,SEX,COHORT,CLINICAL_EVENT,TYPE,SAAMethod,SAA_Status,SAA_Type,InstrumentRep1,InstrumentRep2,InstrumentRep3,RUNDATE,PROJECTID,PI_NAME,PI_INSTITUTION,earliest_date
0,245573,Male,PD,BL,Cerebrospinal Fluid,Amprion-24h alpha-synuclein-SAA,Positive,Type1,2,2,2,2023-12-18,237,Luis Concha,Amprion,2023-12-18
1,163324,Male,PD,BL,Cerebrospinal Fluid,Amprion-24h alpha-synuclein-SAA,Positive,Type1,2,2,2,2023-12-18,237,Luis Concha,Amprion,2023-12-18
2,250240,Male,PD,BL,Cerebrospinal Fluid,Amprion-24h alpha-synuclein-SAA,Positive,Type1,2,2,2,2023-12-18,237,Luis Concha,Amprion,2023-12-18
3,164985,Male,PD,BL,Cerebrospinal Fluid,Amprion-24h alpha-synuclein-SAA,Positive,Type1,2,2,2,2023-12-18,237,Luis Concha,Amprion,2023-12-18
4,245591,Male,PD,BL,Cerebrospinal Fluid,Amprion-24h alpha-synuclein-SAA,Positive,Type1,2,2,2,2023-12-18,237,Luis Concha,Amprion,2023-12-18


In [26]:
# List of columns to drop
columns_to_drop = ['SEX', 'COHORT', 'CLINICAL_EVENT', 'TYPE', 'PI_NAME', 'PI_INSTITUTION']

# Dropping the specified columns from the Biospecimen_Analysis DataFrame
Biospecimen_Analysis_cleaned = Biospecimen_Analysis.drop(columns=columns_to_drop)

# Displaying the first few rows of the cleaned DataFrame
Biospecimen_Analysis_cleaned.head()




,PATNO,SAAMethod,SAA_Status,SAA_Type,InstrumentRep1,InstrumentRep2,InstrumentRep3,RUNDATE,PROJECTID
0,245573,Amprion-24h alpha-synuclein-SAA,Positive,Type1,2,2,2,2023-12-18,237
1,163324,Amprion-24h alpha-synuclein-SAA,Positive,Type1,2,2,2,2023-12-18,237
2,250240,Amprion-24h alpha-synuclein-SAA,Positive,Type1,2,2,2,2023-12-18,237
3,164985,Amprion-24h alpha-synuclein-SAA,Positive,Type1,2,2,2,2023-12-18,237
4,245591,Amprion-24h alpha-synuclein-SAA,Positive,Type1,2,2,2,2023-12-18,237


In [28]:


# Assuming combined_table and Biospecimen_Analysis_Cleaned are already loaded DataFrames

# Performing a left join between combined_table and Biospecimen_Analysis_Cleaned
merged_table = pd.merge(
    combined_table,
    Biospecimen_Analysis_Cleaned,
    left_on='Participant_ID',  # Key from combined_table
    right_on='PATNO',          # Key from Biospecimen_Analysis_Cleaned
    how='left'                 # This specifies a left join
)

# Displaying the first few rows of the merged DataFrame
merged_table.head()


,Participant_ID,COHORT_x,ENROLL_DATE,ENROLL_AGE,AV133STDY,TAUSTDY,GAITSTDY,PISTDY,SV2ASTDY,ENRLPINK1,...,SAA_Status,SAA_Type,InstrumentRep1,InstrumentRep2,InstrumentRep3,RUNDATE,PROJECTID,PI_NAME,PI_INSTITUTION,earliest_date
0,3000,2,02/2011,69.1,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3001,1,03/2011,65.1,0.0,0.0,0.0,0.0,0.0,0.0,...,Positive,NaN,2.0,2.0,2.0,2022-03-17,155.0,Claudia Soto,Amprion,2022-03-17
2,3002,1,03/2011,67.6,0.0,0.0,0.0,0.0,0.0,0.0,...,Positive,NaN,2.0,2.0,2.0,2022-05-12,155.0,Claudia Soto,Amprion,2022-05-12
3,3003,1,04/2011,56.7,0.0,0.0,0.0,0.0,0.0,0.0,...,Positive,NaN,2.0,2.0,2.0,2022-04-14,155.0,Claudia Soto,Amprion,2022-04-14
4,3004,2,04/2011,59.4,0.0,0.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
